## CIFAR-10 zero-shot classifier with Resnet50
In this notebook, we provide a use-case of our method.
We use a Resnet50 model (pre-trained on ImageNet) and leverage Text-To-Concept to turn it into a zero-shot classifer on CIFAR-10.
This notebook has these sections:
+ <i>Preliminaries</i>: we import required libraries and load transformations.
+ <i>Resnet50</i>: we load the model and implement its necessary functions, enabling us to use `TextToConcept` framework.
+ <i>Linear Aligner</i>: we initiate `TextToConcept` object and train/load its linear aligner.
+ <i>Zero-shot classifier</i>: we use methods implemented in `TextToConcept` and appropriate text prompts to get the zero-shot classifer.
+ <i>Zero-shot performance on CIFAR-10</i>: we load CIFAR-10 and evaluate Resnet50-based zero-shot classifier on it. 


### Preliminaries
In this section, we import the required libraries and initialize standard transformations necessary for loading datasets. It is worth mentioning that certain models require input normalization, while others do not.

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
import numpy as np
from tqdm import tqdm
from TextToConcept import TextToConcept

In [40]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [42]:
std_transform_without_normalization = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor()])


std_transform_with_normalization = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)])

### Resnet50
In this part, we load Resnet50 model.
In order to use ``TextToConcept`` framework, model should implement these functions/attributes:
+ ``forward_features(x)`` that takes a tensor as the input and outputs the representation (features) of input $x$ when it is passed through the model.
+ ``get_normalizer`` should be the normalizer that the models uses to preprocess the input. e.g., Resnet18, uses standard ImageNet normalizer.
+ Attribute ``has_normalizer`` should be `True` when normalizer is need for the model.

In [44]:
model = torchvision.models.resnet50(pretrained=True)

encoder = torch.nn.Sequential(*list(model.children())[:-1])
model.forward_features = lambda x : encoder(x)
model.get_normalizer = torchvision.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
model.has_normalizer = True

### Linear Aligner

<b>Initiating Text-To-Concept Object</b><br>
In this section, we initiate ``TextToConcept`` object which turns the vision encoder (e.g., Resnet50) into a model capable of integrating language and vision. By doing so, we enable the utilization of certain abilities present in vision-language models.

In [45]:
text_to_concept = TextToConcept(model)

We can either train the aligner or load an existing one.

#### Training Linear Aligner

<b>Loading ImageNet Dataset to Train the Aligner</b><br>
We note that even $20\%$ of ImageNet training samples suffices for training an effective linear aligner. 
We refer to Appendix A of our paper for more details on sample efficiency of linear alignment.

In [46]:
# loading imagenet dataset to train aligner.
dset = torchvision.datasets.ImageNet(root='/fs/cml-datasets/ImageNet/ILSVRC2012',
                                     split='train',
                                     transform=std_transform_without_normalization)

# 20% of images are fairly enough.
num_of_samples = int(0.02 * len(dset))
dset = torch.utils.data.Subset(dset, np.random.choice(np.arange(len(dset)), num_of_samples, replace=False))

<b>Training the Linear Aligner</b><br>
After loading the object, we need to train the aligner.
+ In order to train the aligner, ``train_linear_aligner`` should be called which obtains representations of the given model (e.g., Resnet50) on ``dset`` as well that of a vision-language model such as CLIP. These representations can also be loaded. Next, this function solves the linear transformation and obtain optimal alignment from model's space to vision-language space.
+ By calling the function ``save_linear_aligner``, linear aliger will be stored which can be utilized later.

In [49]:
path_to_model = '/cmlscratch/mmoayeri/ftr_alignment/saved_ftrs/pytorch_resnet50_500samples_per_class.npy'
path_to_clip_model = '/cmlscratch/mmoayeri/ftr_alignment/saved_ftrs/clip_ViT-B_16_500samples_per_class.npy'

text_to_concept.train_linear_aligner(dset,
                                     load_reps=True,
                                     path_to_model=path_to_model,
                                     path_to_clip_model=path_to_clip_model)

text_to_concept.save_linear_aligner('imagenet_resnet50_aligner.pth')

Loading representations ...
Training linear aligner ...
Linear alignment: ((500000, 2048)) --> ((500000, 512)).
Initial MSE, R^2: 7.130, -0.584
Epoch number, loss: 0, 1.304
Epoch number, loss: 1, 1.196
Epoch number, loss: 2, 1.180
Epoch number, loss: 3, 1.175
Epoch number, loss: 4, 1.173
Final MSE, R^2 = 1.167, 0.741


<b>Loading the Linear Aligner</b><br>
We can also use an already existing linear aligner, to do so, we use the function ``load_linear_aligner``.

In [25]:
text_to_concept.load_linear_aligner('imagenet_resnet50_aligner.pth')

### Zero-shot Classifier
We note that CIFAR-10 is a <i>$10$-way</i> classification problem. 
We use prompts of the form `a pixelated of {c}` to get appropriate concepts in vision-language space.

In [50]:
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [51]:
cifar_zeroshot_classifier = text_to_concept.get_zero_shot_classifier(cifar_classes,
                                                                     prompts=['a pixelated photo of a {}'])

### Zero-shot performance on CIFAR-10
After loading CIFAR-10, we use `cifar_zeroshot_classifier(x)` to get logits of the classification problem when input $x$ is given.

In [52]:
cifar = torchvision.datasets.CIFAR10(root='data/',
                                     download=True,
                                     train=False,
                                     transform=std_transform_with_normalization)

Files already downloaded and verified


In [53]:
loader = torch.utils.data.DataLoader(cifar, batch_size=16, shuffle=True, num_workers=8)
correct, total = 0, 0
with torch.no_grad():
    for data in tqdm(loader):
        x, y = data[:2]
        x = x.to(device)

        outputs = cifar_zeroshot_classifier(x).detach().cpu()
        _, predicted = outputs.max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()

100%|███████████████████████████████████████| 625/625 [00:19<00:00, 32.83it/s]


In [54]:
f'ResNet50 Zeroshot Accuracy on CIFAR-10 {100.*correct/total:.2f}'

'ResNet50 Zeroshot Accuracy on CIFAR-10 67.54'